# Download dlib Lib and unzip the File

In [16]:
!pip install dlib
!pip install wget

!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK


--2023-12-14 01:25:24--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: 'shape_predictor_68_face_landmarks.dat.bz2'

     0K .......... .......... .......... .......... ..........  0%  193K 5m24s
    50K .......... .......... .......... .......... ..........  0%  370K 4m6s
   100K .......... .......... .......... .......... ..........  0%  382K 3m38s
   150K .......... .......... .......... .......... ..........  0%  399K 3m23s
   200K .......... .......... .......... .......... ..........  0% 3.57M 2m46s
   250K .......... .......... .......... .......... ..........  0%  434K 2m42s
   300K .......... .......... .......... .......... ..........  0% 3.52M 2m21s
   350K .......... .......... .......... .......... ..........  0% 3.57M 2m5s
   400K .......... .......... .......... .

# Libraries Import

In [1]:
import torch
import torch.nn as nn
import cv2
import torch
import numpy as np
from torchvision.transforms import ToTensor
import dlib
from sklearn.preprocessing import StandardScaler
import os
import csv

# Facial Extraction

In [2]:

def face_detection(frame, face_cascade):
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    return faces

def facial_landmarks(frame, faces, predictor):
    facial_landmark = []

    for (x, y, w, h) in faces:
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        shape = predictor(frame, rect)
        shape = np.array([(shape.part(i).x, shape.part(i).y) for i in range(shape.num_parts)])
        facial_landmark.append(shape)

    return facial_landmark

def flat_landmarks(facial_landmarks):
    flattened_output = []
    for shape in facial_landmarks:
        flattened_output.extend(shape.flatten())
    return flattened_output

def get_visual_features(cap, face_cascade, predictor):
    framesct = 0
    features = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        framesct += 1
        faces = face_detection(frame, face_cascade)
        facial_landmark = facial_landmarks(frame, faces, predictor)
        flattened_landmarks = flat_landmarks(facial_landmark)
        features += flattened_landmarks

        print(".", end="", flush=True)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Number Of Frames: ", framesct)
    return features

def process_video(video_path, face_cascade, predictor):
    cap = cv2.VideoCapture(video_path)
    features = get_visual_features(cap, face_cascade, predictor)
    cap.release()
    return features

def extract_features_from_directory(directory_path, output_csv, face_cascade, predictor):
    with open(output_csv, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Video"])

        for filename in os.listdir(directory_path):
            if filename.endswith(".mp4"):
                video_path = os.path.join(directory_path, filename)
                print(f"Processing video: {video_path}")
                print("Loading", end="")
                features = process_video(video_path, face_cascade, predictor)
                csv_writer.writerow([filename] + features)
            print("\nLoading complete!")




In [3]:


if __name__ == "__main__":
    # Load the pre-trained models only once
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    video_directory = 'D:/EDU/Graduation Project/Project/Real-life_Deception_Detection_2016/Clips/Deceptive/'
    output_csv_file = 'Facial_Deceptive_features.csv'
    extract_features_from_directory(video_directory, output_csv_file, face_cascade, predictor)

Processing video: D:/EDU/Graduation Project/Project/Real-life_Deception_Detection_2016/Clips/Deceptive/trial_lie_001.mp4
Loading..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Number Of Frames:  510

Loading complete!
Processing video: D:/EDU/Graduation Project/Project/Real-life_Deception_Detection_2016/Clips/Deceptive/trial_lie_002.mp4
Loading................................................................................................................................................................................................

In [4]:
if __name__ == "__main__":
    # Load the pre-trained models only once
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    video_directory = 'D:/EDU/Graduation Project/Project/Real-life_Deception_Detection_2016/Clips/Truthful/'
    output_csv_file = 'Facial_Truthful_features.csv'
    extract_features_from_directory(video_directory, output_csv_file, face_cascade, predictor)

Processing video: D:/EDU/Graduation Project/Project/Real-life_Deception_Detection_2016/Clips/Truthful/trial_truth_001.mp4
Loading............................................................................................................................................................................................................................................................................................................................................................................................................................Number Of Frames:  412

Loading complete!
Processing video: D:/EDU/Graduation Project/Project/Real-life_Deception_Detection_2016/Clips/Truthful/trial_truth_002.mp4
Loading................................................................................................................................................................................................................................................................................................